# iLQR
Location for Linear-Quadratic Regulator (LQR) and iLQR for interactive trajectory planning.

References:
- [1] - [Dewolf - studywolf_control](https://github.com/studywolf/control/blob/master/studywolf_control/controllers/ilqr.py)
- [2] - [Fridovich-Keil - ilqgames](https://github.com/HJReachability/ilqgames/blob/master/python/dynamical_system.py)
- [3] - [Jackson, Howell - iLQR Tutorial](http://roboticexplorationlab.org/papers/iLQR_Tutorial.pdf)
- [4] - [Jackson - AL iLQR Tutorial](https://bjack205.github.io/papers/AL_iLQR_Tutorial.pdf)
- [5] - [Sears-Collins - Linear Quadratic Regulator Python Example](https://automaticaddison.com/linear-quadratic-regulator-lqr-with-python-code-example/)

In [60]:
import sys
sys.path.append('/mnt/c/Users/ZachJW/Documents/Grad/Research/icon/src/')

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_discrete_are

from dynamics import DoubleIntegratorDynamics, UnicycleDynamics, CarDynamics
from cost import QuadraticCost

%matplotlib ipympl

In [61]:
plt.figure()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [62]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def expected_redu(ΔV, α):
    return np.sum(α*ΔV[0,:] + α**2 * ΔV[1,:])

### iLQR

In [63]:
class iLQR(object):
    """iLQR solver"""
    
    # -- Forward Pass --
    GAMMA = 0.5 # scaling factor
    ALPHA_MIN = 1e-6 # scaling bounds
    BETA_RANGE = (1e-4, np.inf) # acceptable cost reduction ratio
    
    # -- Backard Pass
    PHI = 10.0 # regularization scaling
    RHO_MAX = 1e3 # regularization ceiling
    
    # -- iLQR --
    N_LQR_ITER = 20 # number of successful iLQR iterations
    N_LS_ITER = 20 # number of line search iterations
    EPSILON = 1e-3 # convergence criterion
    J_MAX = 1e8 # cost ceiling for forward pass
    
    def __init__(self, dynamics, cost, N=10):
        self.dynamics = dynamics
        self.cost = cost
        self.N = N

        self.cost_hist = None
    
    @property
    def n_x(self):
        """Shorthand to retrieve the number of states."""
        return self.dynamics.n_x
    
    @property
    def n_u(self):
        """Shorthand to retrieve the number of controls."""
        return self.dynamics.n_u
    
    @property
    def dt(self):
        """Shorthand to retrieve time step."""
        return self.dynamics.dt
    
    @property
    def xf(self):
        """Shorthand to retrieve goal state."""
        return self.cost.xf
    
    def rollout(self, x0, U):
        """Rollout the system from an initial state with a control sequence U."""
        
        X = np.zeros((self.N, self.n_x))
        X[0] = x0.copy()
        J = 0.0
        
        for t in range(self.N-1):
            X[t+1] = self.dynamics(X[t], U[t])
            J += self.cost(X[t], U[t])
        J += self.cost(X[-1], U[-1], terminal=True)

        return X, J
    
    def line_search(self, J, ΔV, α):
        """Determine whether to accept the updated trajectory and controls."""
        
        # Compare actual to anticipated cost reduction.
        ΔJ = self.cost_hist[-1] - J
        expected_ΔV = expected_redu(ΔV, α)
        z = ΔJ / -expected_ΔV

        # Reduction in cost means success! Accept new trajectories and reduce the 
        # regularization to converge slower.
        if self.BETA_RANGE[0] <= z and z <= self.BETA_RANGE[1]:
            self.cost_hist.append(J)
            accept = True

        # Increase in cost means failure... Reject new trajectories and increase the
        # regularization to converge quicker.
        else:
            # self.cost_hist.append(self.cost_hist[-1])
            accept = False
            α *= self.GAMMA
            
            # Gate α within some threshold.
            α = max(α, self.ALPHA_MIN)
        
        return accept, α, ΔJ, z
    
    def forward_pass(self, X, U, K, d, ΔV):
        """Forward pass based on algorithm 2 [4]."""
        
        # NOTE: literature says 1.0
        α = 0.5
        
        # Conduct a line search to find a satisfactory trajectory
        for i_ls in range(self.N_LS_ITER):
            X_next = np.zeros((self.N, self.n_x))
            U_next = np.zeros((self.N, self.n_u))

            X_next[0] = X[0].copy()
            J = 0.0

            for t in range(self.N-1):
                δx = X_next[t] - X[t]
                δu = K[t]@δx + α*d[t]
                U_next[t] = U[t] + δu

                X_next[t+1] = self.dynamics(X_next[t], U_next[t])
                J += self.cost(X_next[t], U_next[t])
            J += self.cost(X_next[-1], U_next[-1], terminal=True)
            accept, α, ΔJ, z = self.line_search(J, ΔV, α)
            print(f'\t[FP] {i_ls+1}/{self.N_LS_ITER}\t{J = :.6g}\t{α = :.6g}\t{z = :.6g}') 
            if accept:
                break
        
        return X_next, U_next, J, ΔJ, accept
    
    def backward_pass(self, X, U, ρ=0.0):
        """Backward pass based on algorithm 1 [4]."""
        
        # ρ = 0.0 # DBG
        K = np.zeros((self.N, self.n_u, self.n_x)) # Feedforward gain
        d = np.zeros((self.N, self.n_u)) # Line search gain
        ΔV = np.zeros((self.N, 2)) # two coefficients in expected cost reduction
        
        # Initialize cost jacobian and hessian
        L_x, _, L_xx, _, _ = self.cost.quadraticize(X[-1], U[-1], self.dt)
        p = L_x
        P = L_xx
        
        for t in range(self.N-2, -1, -1):
            L_x, L_u, L_xx, L_uu, L_ux = self.cost.quadraticize(X[t], U[t], self.dt)
            A, B = self.dynamics.linearize(X[t], U[t])
            
            Q_x = L_x + A.T @ p
            Q_u = L_u + B.T @ p
            Q_xx = L_xx + A.T @ P @ A
            Q_uu = L_uu + B.T @ P @ B
            Q_ux = L_ux + B.T @ P @ A
            Q_xu = L_ux.T + A.T @ P @ B
            
            Q_uu += ρ * np.eye(self.n_u)
            Q_uu_inv = np.linalg.inv(Q_uu)
            
            K[t] = -Q_uu_inv @ Q_ux
            d[t] = -Q_uu_inv @ Q_u
            ΔV[t] = np.array([[d[t].T @ Q_u, 0.5*d[t].T @ Q_uu @ d[t]]])
            
            p = Q_x + K[t].T @ Q_uu @ d[t] + K[t].T @ Q_u + Q_xu @ d[t]
            P = Q_xx + K[t].T @ Q_uu @ K[t] + K[t].T @ Q_ux + Q_xu @ K[t]
            
        return K, d, ΔV
    
    def is_verged(self, ΔJ):
        """Determine whether we've solved the OCP to within a threshold or diverged."""
        
        if abs(ΔJ) < self.EPSILON:
            print('Convergence!')
            return True
        elif self.cost_hist[-1] > self.J_MAX:
            print('Divergence...')
            return True
        return False
        
    def run(self, x0, n_lqr_iter=None):
        """Solve the OCP."""
        
        ρ = 0.0 # regularization parameter
        if n_lqr_iter is None:
            n_lqr_iter = self.N_LQR_ITER
        
        # U = np.zeros((self.N, self.n_u))
        U = np.random.randn(self.N, self.n_u) * 1e-6
        X, J = self.rollout(x0, U)
        
        self.cost_hist = [J]
        print(f'[run] initial: {J = :.6g}')        
        
        for i in range(n_lqr_iter):
            # Backward recurse to compute gain matrices.
            K, d, ΔV = self.backward_pass(X, U, ρ)

            # Rollout the full trajectory.
            X_next, U_next, J, ΔJ, accept = self.forward_pass(X, U, K, d, ΔV)

           # Successful iteration, accept the candidate trajectories.
            if accept and J < self.J_MAX:
                # print('[run] Accepted new trajectory!')
                X = X_next
                U = U_next
                ρ = 0.0

            # Forward pass line search failed. Retry backward pass with more regularization.
            else:
                print('[run] Failed line search.. increasing ρ.')
                ρ = max(ρ*self.PHI, 1.0)
                
                # Give up if we've exceeded the regularization ceiling.
                if ρ > self.RHO_MAX:
                    print('[run] Failed regularization.. giving up.')
                    break
            
            print(f'[run] {i+1}/{n_lqr_iter}\tρ: {ρ:.3g}\t{ΔJ = :.6g}')
            if accept and self.is_verged(ΔJ):
                break

        return X, U, J

In [64]:
# x0 = np.array([1.0, 1.0, np.pi/2])
# u = np.array([1.0, 0.0])
# # dynamics(x0, u)
# A, B = dynamics.linearize(x0, u)

In [65]:
# %%prun
dt = 0.2
N = 20
n_lqr_iter = 40

# dynamics = DoubleIntegratorDynamics(dt)
# x0 = np.array([5, 0])
# xf = np.array([0, 0])
# Q = np.eye(2)
# R = np.eye(1)

# dynamics = CarDynamics(dt)
# x0 = np.array([1, 1, 0])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 1])
# R = np.eye(2)

dynamics = UnicycleDynamics(dt)
x0 = np.array([10, 10, 0, -np.pi/2])
xf = np.array([0, 0, 0, np.pi/4])
Q = np.diag([2, 2, 1, 1])
R = np.eye(2)

quad_cost = QuadraticCost(xf, Q, R)
ilqr = iLQR(dynamics, quad_cost, N=N)
X, U, cost = ilqr.run(x0, n_lqr_iter)
dynamics.plot(X, xf)

[run] initial: J = 3920.33
	[FP] 1/20	J = 4279.98	α = 0.25	z = -17.7651
	[FP] 2/20	J = 3566.58	α = 0.25	z = 41.9151
[run] 1/40	ρ: 0	ΔJ = 353.751
	[FP] 1/20	J = 7535.72	α = 0.25	z = -195.104
	[FP] 2/20	J = 2464.7	α = 0.25	z = 136.667
[run] 2/40	ρ: 0	ΔJ = 1101.89
	[FP] 1/20	J = 1470.46	α = 0.5	z = 61.7686
[run] 3/40	ρ: 0	ΔJ = 994.233
	[FP] 1/20	J = 1236.61	α = 0.5	z = 45.8495
[run] 4/40	ρ: 0	ΔJ = 233.857
	[FP] 1/20	J = 1179.26	α = 0.5	z = 46.7674
[run] 5/40	ρ: 0	ΔJ = 57.3445
	[FP] 1/20	J = 1169.25	α = 0.5	z = 25.6862
[run] 6/40	ρ: 0	ΔJ = 10.0083
	[FP] 1/20	J = 1165.97	α = 0.5	z = 35.9639
[run] 7/40	ρ: 0	ΔJ = 3.28673
	[FP] 1/20	J = 1166.45	α = 0.25	z = -8.1883
	[FP] 2/20	J = 1165.46	α = 0.25	z = 22.295
[run] 8/40	ρ: 0	ΔJ = 0.507183
	[FP] 1/20	J = 1166.31	α = 0.25	z = -38.6184
	[FP] 2/20	J = 1165.71	α = 0.125	z = -27.2882
	[FP] 3/20	J = 1165.54	α = 0.0625	z = -19.7769
	[FP] 4/20	J = 1165.49	α = 0.03125	z = -15.3811
	[FP] 5/20	J = 1165.47	α = 0.015625	z = -12.9934
	[FP] 6/20	J = 1165.46	α =

In [46]:
ilqr.cost_hist

[5272.170778913021,
 4765.153250250636,
 3141.5722884928086,
 1648.412114589147,
 1276.6056388792654,
 1182.0314713726505,
 1158.9396544645915,
 1153.3686709083925,
 1153.0565684311541,
 1152.5189387349483,
 1152.0402132821503,
 1151.6726470542662,
 1151.5324380649063]

### LQR

In [22]:
class LQR(iLQR):
    """Linear Quadratic Regulator that assumes linear dynamics and quadratic costs."""
    
    def __init__(self, dynamics, cost, N=10, Q=None, R=None):
        self.cost = cost
        self.dynamics = dynamics
        self.N = N
        
        if Q is None:
            Q = np.eye(self.n_x)
        if R is None:
            R = np.eye(self.n_u)
        self.Q = Q
        self.R = R
        
    def backward_pass(self, X, U):
        """Compute the optimal feedforward gain K."""
        
        K = np.zeros((self.N, self.n_u, self.n_x))
        P = np.zeros((self.N, self.n_x, self.n_x))
        P[-1] = self.Q
        
        for t in range(self.N-2, -1, -1):
            A, B = self.dynamics.linearize(X[t], U[t]) 
            # Feedforward gain [4] (30)
            K[t] = np.linalg.inv(self.R + B.T @ P[t+1] @ B) @ B.T @ P[t+1] @ A
            # Cost-to-go [4] (32)
            P[t] = self.Q + (A.T @ P[t+1] @ A) - A.T @ P[t+1].T @ B @ K[t]
            
            # Q: What's the difference with these two?
            # P[t] = la.solve_discrete_are(A, B, Q, R)
            # K[t] = np.linalg.pinv(R + B.T @ P[t] @ B) @ B.T @ P[t] @ A

        return K
    
    def forward_pass(self, X, U, K):
        """Apply the feedforward gain to compute the next trajectory."""
        
        X_next = np.zeros((self.N, self.n_x))
        U_next = np.zeros((self.N, self.n_u))
        
        X_next[0] = x0
        J = 0.0

        for t in range(self.N-1):
            U_next[t] = -K[t] @ (X_next[t] - self.xf)
            X_next[t+1] = self.dynamics(X_next[t], U_next[t])
            J += self.cost(X_next[t], U_next[t])
        J += self.cost(X_next[-1], U_next[-1], terminal=True)

        return X_next, U_next, J
    
    def run(self, x0, n_iter=10):
        """Solve the LQR OCP."""
        
        U = np.random.randn(self.N, self.n_u) * 1e-3
        X, J0 = self.rollout(x0, U)

        K = self.backward_pass(X, U)
        X, U, Jf = self.forward_pass(X, U, K)
        print(f'J0: {J0:.3g}\tJf: {Jf:.3g}')

        return X, U, Jf

In [24]:
N = 20
dt = 1.0

dynamics = DoubleIntegratorDynamics(dt)
x0 = np.array([5.0, 0.0])
xf = np.array([0.0, 0.0])
Q = np.eye(2)
R = np.eye(1)

# dynamics = CarDynamics(dt)
# x0 = np.array([5.0, 5.0, 0])
# xf = np.array([0, 0, 0])
# Q = np.diag([2.0, 2.0, 1.0])
# R = np.eye(2)

quad_cost = QuadraticCost(xf, Q, R)
lqr = LQR(dynamics, quad_cost, N=N, Q=Q)
X, U, J = lqr.run(x0)
dynamics.plot(X, xf)

J0: 360	Jf: 25
